In [51]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import locale
locale.setlocale(locale.LC_ALL, 'es_ES.utf8')
import warnings
warnings.filterwarnings("ignore")

# ETL Homicidios: Hechos

In [23]:
df_hechos = pd.read_excel('../Data/homicidios.xlsx')
df_victimas = pd.read_excel('../Data/homicidios.xlsx', sheet_name='VICTIMAS')

print(df_hechos.shape)
df_hechos.head()

(696, 21)


,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [24]:
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

In [25]:
df_hechos.describe()

,N_VICTIMAS,FECHA,AAAA,MM,DD,Altura,COMUNA
count,696.000000,696,696.000000,696.000000,696.000000,129.000000,696.000000
mean,1.030172,2018-09-14 05:12:24.827586048,2018.188218,6.692529,15.936782,3336.635659,7.425287
min,1.000000,2016-01-01 00:00:00,2016.000000,1.000000,1.000000,30.000000,0.000000
25%,1.000000,2017-04-02 12:00:00,2017.000000,4.000000,9.000000,1359.000000,4.000000
50%,1.000000,2018-07-12 12:00:00,2018.000000,7.000000,16.000000,2551.000000,8.000000
75%,1.000000,2020-01-04 06:00:00,2020.000000,10.000000,23.000000,4500.000000,11.000000
max,3.000000,2021-12-30 00:00:00,2021.000000,12.000000,31.000000,16080.000000,15.000000
std,0.179393,NaN,1.683754,3.571309,8.639646,3060.641793,4.387050


In [26]:
df_hechos['TIPO_DE_CALLE'].unique()

array(['AVENIDA', 'GRAL PAZ', 'CALLE', 'AUTOPISTA'], dtype=object)

In [27]:
df_hechos.replace(['SD','sd'], np.nan, inplace=True)

In [29]:
df_hechos['HORA'] = pd.to_datetime(df_hechos['HORA'], format='%H:%M:%S').dt.time
df_hechos['FECHA'] = pd.to_datetime(df_hechos['FECHA'])

df_hechos['DIA_SEMANA'] = df_hechos['FECHA'].dt.day_name(locale='es_ES.UTF-8')

In [30]:
df_hechos['FECHA'] = pd.to_datetime(df_hechos['FECHA']).dt.date

In [31]:
df_hechos = df_hechos.drop(['Dirección Normalizada'], axis=1)

df_hechos = df_hechos.drop(['Altura', 'Cruce'], axis=1)

df_hechos.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO,DIA_SEMANA
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,Viernes
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS,Sábado
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7.0,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO,Domingo
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0.0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,NaN,Domingo
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5.0,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS,Jueves


In [32]:
df_hechos.to_excel('../Data/df_hechos.xlsx', index=False)

# ETL Homicidios: Víctimas

In [33]:
print(df_victimas.shape)
df_victimas.head()

(717, 10)


,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [34]:
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [35]:
df_victimas.describe()

,FECHA,AAAA,MM,DD
count,717,717.000000,717.000000,717.000000
mean,2018-09-09 01:36:24.100418304,2018.174338,6.688982,15.960948
min,2016-01-01 00:00:00,2016.000000,1.000000,1.000000
25%,2017-04-01 00:00:00,2017.000000,4.000000,9.000000
50%,2018-07-07 00:00:00,2018.000000,7.000000,16.000000
75%,2019-12-18 00:00:00,2019.000000,10.000000,23.000000
max,2021-12-30 00:00:00,2021.000000,12.000000,31.000000
std,NaN,1.672655,3.575140,8.683358


In [36]:
df_victimas.replace(['SD','sd'], np.nan, inplace=True)

In [37]:
df_victimas['MES'] = df_victimas['FECHA'].dt.month_name(locale='es_ES.UTF-8')

df_victimas['DIA_SEMANA'] = df_victimas['FECHA'].dt.day_name(locale='es_ES.UTF-8')

In [38]:
df_victimas['FECHA'] = pd.to_datetime(df_victimas['FECHA']).dt.date

In [49]:
bins = [0, 21, 41, 61, 123]
labels = ['< 21', '21 - 40', '41 - 60', '> 60']

df_victimas['RANGO_EDAD'] = pd.cut(df_victimas['EDAD'], bins=bins, labels=labels, right=False)

In [40]:
df_victimas.drop(['FECHA_FALLECIMIENTO'], axis=1, inplace=True)

In [50]:
df_victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,MES,DIA_SEMANA,RANGO_EDAD
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19.0,Enero,Viernes,< 21
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70.0,Enero,Sábado,> 60
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30.0,Enero,Domingo,21 - 40
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18.0,Enero,Domingo,< 21
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29.0,Enero,Jueves,21 - 40


In [42]:
df_victimas.to_excel('../Data/df_victimas.xlsx', index=False)

# ETL: poblacion_caba

In [52]:
url = 'https://es.wikipedia.org/wiki/Censo_argentino_de_2022'

response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

tabla = soup.find_all('table', class_='wikitable')[0]

datos = []
filas = tabla.find_all('tr')

for fila in filas[1:]:  
    celdas = fila.find_all(['th', 'td'])
    fila_data = [celda.get_text(strip=True) for celda in celdas]
    datos.append(fila_data)

In [53]:
caba_data = datos[3]

In [54]:
caba_data

['4',
 'Ciudad Autónoma de Buenos Aires',
 '2.890.151',
 '3.121.707',
 '8,0',
 '6,4']

In [55]:
poblacion_2010 = int(caba_data[2].replace('.', '').replace(',', ''))
poblacion_2022 = int(caba_data[3].replace('.', '').replace(',', ''))

In [56]:
años = list(range(2010, 2023))

poblacion_año = [poblacion_2010 + (poblacion_2022 - poblacion_2010) * (anio - 2010) / 12 for anio in años]

poblacion_año = list(map(int, poblacion_año))

In [57]:
data = {'Año': años, 'Población': poblacion_año}
poblacion_caba = pd.DataFrame(data)

In [58]:
poblacion_caba

,Año,Población
0,2010,2890151
1,2011,2909447
2,2012,2928743
3,2013,2948040
4,2014,2967336
5,2015,2986632
6,2016,3005929
7,2017,3025225
8,2018,3044521
9,2019,3063818


In [60]:
poblacion_caba.to_excel('../Data/poblacion_caba.xlsx', index=False)